In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [10]:
info_carros = []
infos = []
df_final = pd.DataFrame()

In [11]:
def coletando_infos_carros(preco_inicial, preco_final, marca):
    l = True
    one = False
    paginas = 2
    
    service = Service()

    options = webdriver.ChromeOptions()

    driver = webdriver.Chrome(service=service, options=options)

    url = f'https://www.socarrao.com.br/brasil/?termo={marca}&particular=0.1&preco={preco_inicial}-{preco_final}'
    driver.get(url)
    #
    length = driver.find_elements(By.CSS_SELECTOR, 'div.card-vehicle')
        
        
    for i in range(len(length)):
        
        carros_infos_1 = driver.find_elements(By.CSS_SELECTOR, 'div.card-vehicle')
        infos.append(carros_infos_1[i].text)
        aux = carros_infos_1[i].text
        
        carros_infos_1[i].click()

        wait = WebDriverWait(driver, 10)
        wait.until(EC.number_of_windows_to_be(paginas))
        all_windows = driver.window_handles

        driver.switch_to.window(all_windows[paginas-1])
        wait = WebDriverWait(driver, 10)
        paginas+=1
        html_da_nova_pagina = driver.page_source
        html_da_nova_pagina = driver.find_elements(By.CSS_SELECTOR, 'ul.details')
        auxiliar = html_da_nova_pagina[0].text.split('\n')
        info_carros.append(html_da_nova_pagina[0].text.split('\n'))

        a = aux.split('\n')
            
        meu_dict = {}

        # Iterando sobre a lista e criando o dicionário
        for i in range(0, len(auxiliar), 2):
                chave = auxiliar[i]  # A chave é o elemento atual da lista
                valor = auxiliar[i + 1]  # O valor é o próximo elemento da lista
                meu_dict[chave] = valor
                
        meu_dict.pop('Placa', None)
        if l == True:
            df = pd.DataFrame([meu_dict])
            df['Combustivel'] = a[5]
            j = a[0].split(' ')
            df['Modelo'] = j[1]
            df['Marca'] = j[0]
            df['Preço'] = a[7]
            match = re.search(r'\d+\.\d+', a[1])
            if match != None:
                df['Cilindadras'] = match.group(0)
            else:
                df['Cilindadras'] = a[1]
            l =  False
        else:
            df.loc[len(df)] = meu_dict
            j = a[0].split(' ')
            df['Modelo'] = j[1]
            df['Marca'] = j[0]
            df['Preço'] = a[7]
            match = re.search(r'\d+\.\d+', a[1])
            if match != None:
                df['Cilindadras'] = match.group(0)
            else:
                df['Cilindadras'] = a[1]
            l = False 
        driver.switch_to.window(all_windows[0])
       
    return df
    driver.quit()

In [12]:
marcas = ['BMW','Toyota','Fiat','Mercedes']
precos = [0,20000,40000,60000,80000,100000]
df = pd.DataFrame()
for i in marcas:
    for j in precos:
        df = coletando_infos_carros(j,j+20000,i)
        df_final = pd.concat([df_final, df], ignore_index=True)
        df_final.drop_duplicates()

In [17]:
df = df_final.drop_duplicates()
df

,Ano,Quilometragem,Câmbio,Combustível,Cor,Portas,Carroceria,Combustivel,Modelo,Marca,Preço,Cilindadras
0,2010/2011,0km,Câmbio Manual,Gasolina,Vermelha,-,-,Gasolina,X3,BMW,R$ 15.000,2.5
1,2005/2006,139.004km,Automático,Gasolina,Cinza,4 portas,Coupe,NaN,X3,BMW,R$ 15.000,2.5
2,2014/2015,93.000km,Automático,Gasolina,Branca,4 portas,SUV Grande,NaN,X3,BMW,R$ 15.000,2.5
3,2004/2005,225.000km,Automático,Gasolina,Prata,4 portas,Sedan Medio,NaN,X3,BMW,R$ 15.000,2.5
4,2021/2021,0km,Câmbio Manual,Gasolina,Branca,-,SUV Medio,NaN,X3,BMW,R$ 15.000,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...
324,2018/2018,80.000km,Automático,Flex,Preta,4 portas,SUV Medio,NaN,Gla,Mercedes,R$ 117.999,1.6
325,2014/2015,74.534km,Automático,Gasolina,Preta,4 portas,Sedan Medio,NaN,Gla,Mercedes,R$ 117.999,1.6
326,2015/2015,126.000km,Automático,Gasolina,Branca,4 portas,SUV Medio,NaN,Gla,Mercedes,R$ 117.999,1.6
327,2015/2015,55.300km,Automático,Gasolina,Preta,4 portas,SUV Medio,NaN,Gla,Mercedes,R$ 117.999,1.6


In [18]:
df.to_excel('Cars_infos.xlsx', index=False)

In [19]:
df.head(50)

,Ano,Quilometragem,Câmbio,Combustível,Cor,Portas,Carroceria,Combustivel,Modelo,Marca,Preço,Cilindadras
0,2010/2011,0km,Câmbio Manual,Gasolina,Vermelha,-,-,Gasolina,X3,BMW,R$ 15.000,2.5
1,2005/2006,139.004km,Automático,Gasolina,Cinza,4 portas,Coupe,NaN,X3,BMW,R$ 15.000,2.5
2,2014/2015,93.000km,Automático,Gasolina,Branca,4 portas,SUV Grande,NaN,X3,BMW,R$ 15.000,2.5
3,2004/2005,225.000km,Automático,Gasolina,Prata,4 portas,Sedan Medio,NaN,X3,BMW,R$ 15.000,2.5
4,2021/2021,0km,Câmbio Manual,Gasolina,Branca,-,SUV Medio,NaN,X3,BMW,R$ 15.000,2.5
12,2019/2020,4.100km,Câmbio Manual,Gasolina,Preta,-,Hatch,Gasolina,G310,BMW,Simular financiamento,2021 / 2022
13,1997/1998,125.000km,Automático,Gasolina,Prata,4 portas,Hatchback Medio,NaN,G310,BMW,Simular financiamento,2021 / 2022
14,2001/2002,209.000km,Automático,Gasolina,Prata,4 portas,Coupe,NaN,G310,BMW,Simular financiamento,2021 / 2022
15,2000/2001,106.000km,Automático,Gasolina,Preta,4 portas,Sedan Medio,NaN,G310,BMW,Simular financiamento,2021 / 2022
16,2010/2011,70.000km,Câmbio Manual,Flex,Prata,-,-,NaN,G310,BMW,Simular financiamento,2021 / 2022
